#### asyncio: don't forget to await everything - it's almost a generator

In [ ]:
import asyncio

async def worker(name, queue, to_consume, semaphore):
    async with semaphore:
        print(f'worker {name} consuming {to_consume}')
        await asyncio.sleep(1)
        l = len(to_consume)
        if l == 1:
            return
        await queue.put(to_consume[:l//2])
        await queue.put(to_consume[l//2:])
    
async def coordinator():
    queue = asyncio.Queue()
    semaphore = asyncio.Semaphore(5)
    await queue.put('abcdefghijklmn')
    while True:
        to_consume = await queue.get()
        if to_consume:
            asyncio.create_task(worker('', queue, to_consume, semaphore))

await coordinator()

#### Threads: Remember your locks

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor
from queue import Queue

def worker(to_consume, queue):
    print(f'consuming {to_consume}')
    time.sleep(1)
    l = len(to_consume)
    if l == 1:
        return
    queue.put(to_consume[:l//2])
    queue.put(to_consume[l//2:])

def coordinator():
    pool = ThreadPoolExecutor(5)
    q = Queue()
    q.put('abcdefghijklmn')
    while True:
        to_consume = q.get()
        pool.submit(worker, to_consume, q)

coordinator()

#### asyncio client/server

In [3]:
import asyncio, threading

async def handle_echo(reader, writer):
    data = await reader.read(100)
    message = data.decode()
    addr = writer.get_extra_info('peername')

    print(f"Received {message!r} from {addr!r}")

    print(f"Send: {message!r}")
    writer.write(data)
    await writer.drain()

    print("Close the connection")
    writer.close()
    await writer.wait_closed()

async def main():
    server = await asyncio.start_server(
        handle_echo, '127.0.0.1', 8888)

    addrs = ', '.join(str(sock.getsockname()) for sock in server.sockets)
    print(f'Serving on {addrs}')

    async with server:
        await server.serve_forever()

def server_thread():
    loop = asyncio.new_event_loop()
    def run_server(loop: asyncio.BaseEventLoop):
        asyncio.set_event_loop(loop)
        asyncio.run(main())
    thread = threading.Thread(target=run_server, args=[loop])
    return thread

thread = server_thread()
thread.start()

Serving on ('127.0.0.1', 8888)


code from https://docs.python.org/3/library/asyncio-stream.html#tcp-echo-server-using-streams  
note for serve_forever to work correctly in ipynb either need to not be in the same event loop: use nest_asyncio or a new event loop

In [5]:
import asyncio

async def tcp_echo_client(message):
    reader, writer = await asyncio.open_connection(
        '127.0.0.1', 8888)

    print(f'Send: {message!r}')
    writer.write(message.encode())
    await writer.drain()

    data = await reader.read(100)
    print(f'Received: {data.decode()!r}')

    print('Close the connection')
    writer.close()
    await writer.wait_closed()

await tcp_echo_client('Hello World!')

Send: 'Hello World!'
Received: 'Hello World!'
Close the connection


Received 'Hello World!' from ('127.0.0.1', 50830)
Send: 'Hello World!'
Close the connection


#### Threaded version using sockets

In [8]:
import socket
import threading
 
print_lock = threading.Lock()
 
def threaded(c):
    while True:
        data = c.recv(1024)
        if not data:
            with print_lock:
                print('Bye')
            break
        data = data[::-1]
        c.send(data)
    c.close()
 
 
def server():
    host = '127.0.0.1'

    port = 12345
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((host, port))
    print("socket binded to port", port) 
    s.listen(5) # backlog size
    print("socket is listening")
 
    try:
        while True: 
            c, addr = s.accept()
            with print_lock:
                print('Connected to :', addr[0], ':', addr[1])
            threading.Thread(target=threaded, args=(c,)).start()
    except KeyboardInterrupt:
        s.close()

threading.Thread(target=server).start()

socket binded to port 12345
socket is listening


In [9]:
import socket

def client():
    host = '127.0.0.1'
    port = 12345

    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    s.connect((host,port))

    message = "hello, world!"
    s.send(message.encode('ascii'))
    data = s.recv(1024)
    print('Received from the server :',str(data.decode('ascii')))
    s.close()

client()

Connected to : 127.0.0.1 : 58924
Bye


Received from the server : !dlrow ,olleh
